In [7]:
import json
import pandas as pd
 

# Ler o JSON
with open('./pokedex-db.json', "r", encoding="utf-8") as f:
    pokedex_data = json.load(f)

# Converter para DataFrame
df = pd.DataFrame(pokedex_data)

print(f"Tamanho do DataFrame: {df.shape}\n\n")

 

Tamanho do DataFrame: (1025, 14)




In [9]:
# Removendo colunas desnecessárias se existirem
if 'ThumbnailAltText' in df.columns:
    df.drop(columns=['ThumbnailAltText'], inplace=True)
if 'detailPageURL' in df.columns:
    df.drop(columns=['detailPageURL'], inplace=True)
if 'collectibles_slug' in df.columns:
    df.drop(columns=['collectibles_slug'], inplace=True)
if 'featured' in df.columns:
    df.drop(columns=['featured'], inplace=True)
if 'slug' in df.columns:
    df.drop(columns=['slug'], inplace=True)
if 'ThumbnailImage' in df.columns:
    df.drop(columns=['ThumbnailImage'], inplace=True)
    
# Ordenando colunas
df = df[['id', 'number', 'name', 'type', 'height', 'weight', 'abilities', 'weakness']]

print(f"Tamanho do DataFrame após remoção de colunas: {df.shape}\n\n")
print(df.head(5))
        

Tamanho do DataFrame após remoção de colunas: (1025, 8)


   id number        name             type  height  weight   abilities  \
0   1   0001   Bulbasaur  [grass, poison]     0.7     6.9  [Overgrow]   
1   2   0002     Ivysaur  [grass, poison]     1.0    13.0  [Overgrow]   
2   3   0003    Venusaur  [grass, poison]     2.0   100.0  [Overgrow]   
3   4   0004  Charmander           [fire]     0.6     8.5     [Blaze]   
4   5   0005  Charmeleon           [fire]     1.1    19.0     [Blaze]   

                       weakness  
0  [Fire, Ice, Flying, Psychic]  
1  [Fire, Ice, Flying, Psychic]  
2  [Fire, Ice, Flying, Psychic]  
3         [Water, Ground, Rock]  
4         [Water, Ground, Rock]  


In [75]:
from bs4 import BeautifulSoup
import json

# Abra e leia o arquivo HTML
with open('pokemondb-net.html', 'r', encoding='utf-8') as f:
    html = f.read()

soup = BeautifulSoup(html, 'html.parser')

# Encontre a tabela com id="pokedex"
table = soup.find('table', {'id': 'pokedex'})

# Pegue o cabeçalho da tabela
headers = [th.get_text(strip=True) for th in table.find('thead').find_all('th')]

# Descubra o índice da coluna 'Type'
type_index = headers.index('Type')

# Remova 'Type' dos headers
headers.pop(type_index)

# Extraia os dados das linhas
data = []
for row in table.find('tbody').find_all('tr'):
    cols = row.find_all('td')
    if not cols:
        continue

    # Extrai os textos das colunas, exceto 'Type'
    col_texts = [td.get_text(strip=True) for td in cols]
    col_texts.pop(type_index)

    # Tratamento especial para a coluna 'Name'
    name_td = cols[headers.index('Name')]
    name_a = name_td.find('a')
    name = name_a.get_text(strip=True) if name_a else name_td.get_text(strip=True)
    form_small = name_td.find('small')
    form = form_small.get_text(strip=True) if form_small else None

    # Substitui o valor da coluna 'Name' pelo nome extraído
    col_texts[headers.index('Name')] = name

    # Cria o dicionário
    item = dict(zip(headers, col_texts))
    if form:
        item['form'] = form
    data.append(item)
    
# Remover os pokemons repetidos mantendo apenas o que primeiro aparece na lista
unique_data = []
seen_ids = set()
for item in data:
    poke_id = item.get("#")
    if poke_id and poke_id not in seen_ids:
        unique_data.append(item)
        seen_ids.add(poke_id)
 
# Salvar em um arquivo JSON
with open('pokemon_stats_db.json', 'w', encoding='utf-8') as f:
    json.dump(unique_data, f, ensure_ascii=False, indent=4)

# Concatenar os dados de 'attack', 'defense', 'hp', 'sp_atk', 'sp_def', 'speed' ao DataFrame df comparando o "id" do df com o "#" do unique_data
for item in unique_data:
    poke_id = item.get("#")
    if poke_id and poke_id.isdigit():
        poke_id = int(poke_id)
        df.loc[df['id'] == poke_id, ['attack', 'defense', 'hp', 'sp_atk', 'sp_def', 'speed', 'total']] = [
            item.get('Attack'),
            item.get('Defense'),
            item.get('HP'),
            item.get('Sp. Atk'),
            item.get('Sp. Def'),
            item.get('Speed'),
            item.get('Total')
        ]
         
print(f"Número de linhas extraídas: {len(unique_data)}")
print(df.head())  # Exibir as primeiras linhas do DataFrame atualizado

Número de linhas extraídas: 1025
    abilities  weight                      weakness number  height  \
0  [Overgrow]     6.9  [Fire, Ice, Flying, Psychic]   0001     0.7   
1  [Overgrow]    13.0  [Fire, Ice, Flying, Psychic]   0002     1.0   
2  [Overgrow]   100.0  [Fire, Ice, Flying, Psychic]   0003     2.0   
3     [Blaze]     8.5         [Water, Ground, Rock]   0004     0.6   
4     [Blaze]    19.0         [Water, Ground, Rock]   0005     1.1   

         name  id             type attack defense  hp sp_atk sp_def speed  \
0   Bulbasaur   1  [grass, poison]     49      49  45     65     65    45   
1     Ivysaur   2  [grass, poison]     62      63  60     80     80    60   
2    Venusaur   3  [grass, poison]     82      83  80    100    100    80   
3  Charmander   4           [fire]     52      43  39     60     50    65   
4  Charmeleon   5           [fire]     64      58  58     80     65    80   

  total  
0   318  
1   405  
2   525  
3   309  
4   405  


In [ ]:
import requests
import time

# Adicionar coluna para localizações de captura
df['capture_location'] = None

# Resgatando Localizações de Captura 
def get_capture_locations(number):
    url = f"https://pokeapi.co/api/v2/pokemon/{number}/encounters"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            print(f"Requisição bem-sucedida para o Pokémon número {number}.")
        
            # Extrai o nome dos locais de captura
            locations = [loc['location_area']['name'] for loc in data]
            return locations
        else:
            return []
    except Exception as e:
        return []
    
# Para cada item da base de dados, resgata as localizações de captura
for index, row in df.iterrows():
    number = row['id']   
    print(f"Resgatando localizações de captura para o Pokémon número {number}/{len(df.index)}...")
    locations = get_capture_locations(number)
    print(f"Localizações de captura para o Pokémon número {number}: {locations}")
    df.at[index, 'capture_location'] = locations
    # time.sleep(1)  # Respeitar o limite de requisições
    
# Salvar o DataFrame atualizado em um novo arquivo JSON
df.to_json('./pokedex-db-with-capture-locations.json', orient='records', indent=4, force_ascii=False)
 
print("Arquivo JSON atualizado salvo como 'pokedex-db-with-capture-locations.json'.")
print(f"Tamanho do DataFrame atualizado: {df.shape}")

In [ ]:
# One-hot encoding para cada campo de array

# Garante que os campos de array estejam como listas (caso estejam ausentes)
for col in ['abilities', 'type', 'weakness', 'capture_location']:
    df[col] = df[col].apply(lambda x: x if isinstance(x, list) else [])


# Extrair valores únicos de habilidades, tipos, fraquezas e localizações de captura
abilities_dummies = df['abilities'].explode().str.strip().str.lower().str.title().dropna().unique()
type_dummies = df['type'].explode().str.strip().str.lower().str.title().dropna().unique()
weakness_dummies = df['weakness'].explode().str.strip().str.lower().str.title().dropna().unique()
capture_location_dummies = df['capture_location'].explode().str.strip().str.lower().str.title().dropna().unique()

# Para cada habilidade, tipo, fraqueza e localização de captura, cria colunas one-hot verificando se o valor está presente
df_abilities = df['abilities'].apply(lambda x: pd.Series({f'ability_{a}': int(a in x) for a in abilities_dummies}))
df_type = df['type'].apply(lambda x: pd.Series({f'type_{t}': int(t in x) for t in type_dummies}))
df_weakness = df['weakness'].apply(lambda x: pd.Series({f'weakness_{w}': int(w in x) for w in weakness_dummies}))
df_capture_location = df['capture_location'].apply(lambda x: pd.Series({f'location_{loc}': int(loc in x) for loc in capture_location_dummies}))

# Concatenar tudo
df_final = pd.concat([
    df.drop(['abilities', 'type', 'weakness', 'capture_location'], axis=1),
    df_abilities,
    df_type,
    df_weakness,
    df_capture_location
], axis=1)

# Salvar em CSV
df_final.to_csv('pokedex-db-parsed.csv', index=False, encoding='utf-8')

print("CSV com one-hot encoding gerado com sucesso!")
print(f"Tamanho do DataFrame final: {df_final.shape}\n\n")
print(df_final[['id', 'name'] + list(df_abilities.columns) + list(df_type.columns) +
                list(df_weakness.columns) + list(df_capture_location.columns)])

CSV com one-hot encoding gerado com sucesso!
Tamanho do DataFrame final: (1025, 1188)


        id          name  ability_Overgrow  ability_Blaze  ability_Torrent  \
0        1     Bulbasaur                 1              0                0   
1        2       Ivysaur                 1              0                0   
2        3      Venusaur                 1              0                0   
3        4    Charmander                 0              1                0   
4        5    Charmeleon                 0              1                0   
...    ...           ...               ...            ...              ...   
1020  1021   Raging Bolt                 0              0                0   
1021  1022  Iron Boulder                 0              0                0   
1022  1023    Iron Crown                 0              0                0   
1023  1024     Terapagos                 0              0                0   
1024  1025     Pecharunt                 0            